Sistema de Recomendação


https://www.kaggle.com/datasets/siddharthmandgi/tripadvisor-restaurant-recommendation-data-usa

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy import stats
import scipy.stats as st

In [ ]:
df = pd.read_csv('/content/TripAdvisor_RestauarantRecommendation.csv')

In [ ]:
df.sample(10)

,Name,Street Address,Location,Type,Reviews,No of Reviews,Comments,Contact Number,Trip_advisor Url,Menu,Price_Range
1876,Bill's Cafe on the Alameda,2089 The Alameda,"San Jose, CA 95126-1136","American, Cafe, Diner",4.5 of 5 bubbles,155 reviews,NaN,+1 408-244-9085,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1802,Xian Famous Food,4128 Main St Bsmt 36,"Flushing, NY 11355-3177","Chinese, Fast Food, Asian",4 of 5 bubbles,87 reviews,If you have an appetite for spicy food and nod...,+1 347-294-0022,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$
403,King's Blu Jam Cafe,4711 Louetta Rd,"Spring, TX 77388-4351","Cafe, Cajun & Creole",5 of 5 bubbles,16 reviews,Had a family member tell about this place. Th...,+1 713-389-5555,https://www.tripadvisor.com//Restaurant_Review...,https://img1.wsimg.com/blobby/go/7d01b5de-2694...,$$ - $$$
398,Toulouse Petit Kitchen & Lounge,601 Queen Anne Ave N,"Seattle, WA 98109-4013","American, Cajun & Creole, Vegetarian Friendly",4.5 of 5 bubbles,"1,578 reviews",I loved everything about this restaurant. Like...,+1 206-432-9069,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
2772,Saya Restaurant,8455 S 212th St,"Kent, WA 98031-1950","Asian, Thai, Vegetarian Friendly",4.5 of 5 bubbles,27 reviews,The service was quick and friendly. It seemed ...,+1 253-395-7987,https://www.tripadvisor.com//Restaurant_Review...,https://openmenu.com/restaurant/57a0755e-ebff-...,$
1307,Dinosaur Bar-B-Que,224 Market St,"Newark, NJ 07102-3708","American, Bar, Barbecue",4 of 5 bubbles,506 reviews,My party of 10 people came here on Saturday th...,+1 862-214-6100,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1124,The Grange Bar & Eatery,1635 Amsterdam Ave,"New York City, NY 10031-6103","American, Bar, Vegetarian Friendly",4.5 of 5 bubbles,190 reviews,I went at the grange bar with friends in a col...,+1 212-491-1635,https://www.tripadvisor.com//Restaurant_Review...,https://thegrangebarnyc.com/new-york-hamilton-...,$$ - $$$
72,Red Apple Cafe,488 W Herndon Ave,"Fresno, CA 93650-1329","American, Cafe, Vegetarian Friendly",4.5 of 5 bubbles,87 reviews,NaN,+1 559-261-1505,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
2212,Peyrassol Cafe,1083 Lake Washington Blvd N Ste 30,"Renton, WA 98056-6459","European, French, Vegetarian Friendly",4.5 of 5 bubbles,183 reviews,My husband and I love this restaurant. Everyth...,+1 425-282-5472,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1994,Ponzio's Diner-Bakery-Bar,7 Marlton Pike W,"Cherry Hill, NJ 08002-3098","American, Diner, Vegetarian Friendly",4 of 5 bubbles,381 reviews,I enjoyed my meal at Ponzio’s. It has a wide v...,+1 856-428-4808,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3062 entries, 0 to 3061
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              3062 non-null   object
 1   Street Address    3062 non-null   object
 2   Location          3062 non-null   object
 3   Type              3049 non-null   object
 4   Reviews           3062 non-null   object
 5   No of Reviews     3062 non-null   object
 6   Comments          2447 non-null   object
 7   Contact Number    3062 non-null   object
 8   Trip_advisor Url  3062 non-null   object
 9   Menu              3062 non-null   object
 10  Price_Range       3062 non-null   object
dtypes: object(11)
memory usage: 263.3+ KB


In [ ]:
df['Comments'].sample(10)

3025                                                  NaN
1277    As a 58 year old Greek, I can say this restaur...
1002    Good German food and better German beer, large...
2576    First off, shout out to the best bar tender I'...
1999    It is a very tiny place and I guess in season ...
368                                                   NaN
2009    This was our first time dining in an authentic...
1973    Reservations for V Day were unavailable so we ...
1398    The restaurant has changed.  I think it’s a po...
31      I can't tell you enough of how amazing the foo...
Name: Comments, dtype: object

In [ ]:
df.isna().sum()

Name                  0
Street Address        0
Location              0
Type                 13
Reviews               0
No of Reviews         0
Comments            615
Contact Number        0
Trip_advisor Url      0
Menu                  0
Price_Range           0
dtype: int64

In [ ]:
df2 = df.copy()

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,3),min_df=3,analyzer='word')

#Replace NaN with an empty string
df2['Comments'] = df2['Comments'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['Comments'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(3062, 4435)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
print(cosine_sim.shape)
print(cosine_sim[1])

(3062, 3062)
[0.         1.         0.0370547  ... 0.02827601 0.         0.03051351]


In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df2.index, index=df2['Name']).drop_duplicates()


In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    Trip_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2['Name'].iloc[Trip_indices]

In [ ]:
# Getting the recommendation
get_recommendations('The Left Bank')

1992       Mikado Japanese Cuisine
785           Julio's Mexican Food
1538                   Kenko Sushi
1596           Cannelle Patisserie
674           Lange's Delicatessen
627     Dusal's Italian Restaurant
2458      The Original Blanco Cafe
269                S & D Oyster Co
831          Vila Verde Restaurant
177                Los Tacos No. 1
Name: Name, dtype: object

In [ ]:
df2['Name']